In [19]:
import torch 
import torch.nn as nn

import config

import time 
import copy
from tqdm.notebook import trange, tqdm

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.transforms as transformss

from models.MobileNetV3 import MobileNetV3

from models.functions import train, evaluate, confusion
#importing comet
from comet_ml import Experiment

In [20]:
#Initialising GereralTorch class

#Setting the experiment with the API key stored in config.py
experiment = Experiment(project_name = "MobileNet", workspace = "leothesouthafrican", api_key = config.api_key)

#Report multiple hyperparameters using a dictionary
hyper_params = {
    'learning_rate': 0.001,
    'num_epochs': 30,
    'batch_size': 32,
    'image_size': 32,
    'image_channels': 3,
    'output_size': 10,
    'num_layers': 'na',
    'train_val_split': 0.90,
    'device': 'mps',
    'model_name': 'Basic MobileNetV3',
    'criterion': 'CrossEntropyLoss',
    'optimizer': 'Adam',
    'dataset': 'CIFAR10',
    'best_model_path': 'MN3Small.pt',
}

#Setting the device
device = torch.device(hyper_params['device'])

# Loading model
model = MobileNetV3(mode='small')
model.to(device)

# Setting the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device) #Setting the loss function
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate']) #Setting the optimizer

# Adding model parameters to comet
for name, param in model.named_parameters():
    hyper_params[name] = param

# Logging the hyperparameters to comet
experiment.log_parameters(hyper_params)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/leothesouthafrican/mobilenet/bc92d88d1297442bb7fa2dfbd5b2b3a8



---Model Summary---
MobileNetV3(
  (first_conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): h_swish(
      (sigmoid): h_sigmoid(
        (relu): ReLU6(inplace=True)
      )
    )
  )
  (layers): Sequential(
    (0): InvertedResidualBlock(
      (conv): Sequential(
        (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (depth_conv): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (squeeze_block): SqueezeBlock(
        (flatten): AdaptiveAvgPool2d(output_size=1)
        (dense): Sequential(
          (0): Linear(in_features

In [21]:
#defining transforms

train_transform = transforms.Compose([
                transforms.Resize((hyper_params['image_size'],hyper_params['image_size'])),
                transforms.ToTensor(),
                transforms.RandomRotation(5),
                transforms.RandomHorizontalFlip(0.2),
                transforms.RandomVerticalFlip(0.2),
                transforms.RandomErasing(0.1),
                transforms.RandomCrop(hyper_params['image_size'], padding=2),
            ])

test_transform = transforms.Compose([
                transforms.Resize((hyper_params['image_size'],hyper_params['image_size'])),
                transforms.ToTensor()
            ])

In [22]:
#Loading the CIFAR10 dataset


train_dataset = datasets.CIFAR10(root='data', train=True, transform=train_transform, download=True)
test_dataset = datasets.CIFAR10(root='data', train=False, transform=test_transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=True,
                                            num_workers=4)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=False,
                                            num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
#Splitting the train data into train and validation
n_train = int(len(train_dataset) * hyper_params['train_val_split'])
n_val = len(train_dataset) - n_train

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [n_train, n_val])

val_data = copy.deepcopy(val_dataset)
val_data.dataset.transform = test_transform #change the transform to test_transform to get the validation data in the same format as the test data

#defining the val loader
val_loader = torch.utils.data.DataLoader(val_data,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=False)
                                            

print('Train data size: ', len(train_dataset))
print('Validation data size: ', len(val_dataset))
print('Test data size: ', len(test_dataset))

Train data size:  45000
Validation data size:  5000
Test data size:  10000


In [24]:
with experiment.train():
    train(hyper_params['num_epochs'], model, criterion, optimizer, train_loader, val_loader, hyper_params['best_model_path'], device, experiment)

Begin training...


100%|██████████| 157/157 [00:03<00:00, 39.49it/s]


	Train Loss: 1.812 | Train Acc: 43.75%
Epoch: 02 | Epoch Time: 3m 31s
	 Val. Loss: 1.508 |  Val. Acc: 0.46%


100%|██████████| 157/157 [00:03<00:00, 48.72it/s]


	Train Loss: 1.095 | Train Acc: 56.25%
Epoch: 03 | Epoch Time: 3m 9s
	 Val. Loss: 1.467 |  Val. Acc: 0.46%


...

	Train Loss: 0.929 | Train Acc: 56.25%
Epoch: 29 | Epoch Time: 3m 9s
	 Val. Loss: 0.737 |  Val. Acc: 0.74%


100%|██████████| 157/157 [00:03<00:00, 49.73it/s]


	Train Loss: 1.330 | Train Acc: 50.00%
Epoch: 30 | Epoch Time: 3m 12s
	 Val. Loss: 0.680 |  Val. Acc: 0.76%


100%|██████████| 157/157 [00:03<00:00, 50.20it/s]

	Train Loss: 0.934 | Train Acc: 62.50%
Epoch: 31 | Epoch Time: 3m 14s
	 Val. Loss: 0.669 |  Val. Acc: 0.76%


In [25]:
with experiment.test():

    model.load_state_dict(torch.load(hyper_params['best_model_path']))

    test_loss, test_acc = evaluate(model, test_loader, criterion, device, experiment)

    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

100%|██████████| 313/313 [00:06<00:00, 49.67it/s]

Test Loss: 0.936 | Test Acc: 67.53%


In [26]:
confusion(model, test_loader, device = device, experiment=experiment)

In [27]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/leothesouthafrican/mobilenet/bc92d88d1297442bb7fa2dfbd5b2b3a8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_acc [313]      : (0.27955271565495204, 67.53194888178913)
COMET INFO:     test_loss [313]     : (0.001735667070260825, 0.9361007741084114)
COMET INFO:     train_acc [46890]   : (0.0, 0.96875)
COMET INFO:     train_loss [46890]  : (0.2313055396080017, 3.6640939712524414)
COMET INFO:     train_val_acc [30]  : (0.4596, 0.7636)
COMET INFO:     train_val_loss [30] : (0.6687131062814384, 1.5075855710703856)
COMET INFO:   Parameters:
COMET INFO:     batch_size                             : 32
COMET INFO:     best_model_path                        : MN3Small.pt
COMET INFO:     criterion                              : CrossEntropyLo

In [ ]:
!touch requirements.txt